<a href="https://colab.research.google.com/github/GunnakkagariRajesh/Coding_Raja_01/blob/main/Rajesh_movie_recommendation_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
// MOVIE RECOMENDATION SYSTEM


import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'movielens-20m-dataset:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F339%2F77759%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240309%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240309T153409Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Da6598635923c699fc9260bb377af9607fce971cdff11434e51d3833bd2cd72151c1a6221542bfede67212a272ec05b90f4008b6f36537ed22f035e4904f3317cb3d27ad4f4ec7a1a18b60a05235fcaec987acd21cc28c9ae2735bdae92baac5b9503e8d1778c97acbcc5c52630085d9eff1c9fda94e766f9aab6649c413070739cf13d57e3d6b34c1936bb7fdabbee93bfbe321e16a3c7c3d3ac68bb4756e3780b381fabfb19592aeae1059ffdf5ba1af444430e6168503d2f67a0a952155ebd8ef37712d1d4c1d29be57071e282ee07dcbb13b925ac72f87de5b2ce3a6a5401fdfd6949ec764a45ea57332510fb2b0fe052e1158628548c37239276266e9701'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')

[==================================================] 204953792 bytes downloaded
Downloaded and uncompressed: movielens-20m-dataset
Data source import complete.


In [ ]:


import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



/kaggle/input/movielens-20m-dataset/tag.csv
/kaggle/input/movielens-20m-dataset/movie.csv
/kaggle/input/movielens-20m-dataset/genome_scores.csv
/kaggle/input/movielens-20m-dataset/genome_tags.csv
/kaggle/input/movielens-20m-dataset/link.csv
/kaggle/input/movielens-20m-dataset/rating.csv


In [ ]:
movie = pd.read_csv('../input/movielens-20m-dataset/movie.csv')
rating = pd.read_csv('../input/movielens-20m-dataset/rating.csv')
df = movie.merge(rating, how="left", on="movieId")
df.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.0,4.0,1999-12-11 13:36:47
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,6.0,5.0,1997-03-13 17:50:52
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,8.0,4.0,1996-06-05 13:37:51
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,10.0,4.0,1999-11-25 02:44:47
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,11.0,4.5,2009-01-02 01:13:41


In [ ]:
movie.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
rating.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,2005-04-02 23:53:47
1,1,29,3.5,2005-04-02 23:31:16
2,1,32,3.5,2005-04-02 23:33:39
3,1,47,3.5,2005-04-02 23:32:07
4,1,50,3.5,2005-04-02 23:29:40


In [ ]:
print('Number of Users: {}, Number of movies:{}'  .format(len(df.userId.unique()),len(df.title.unique())))

Number of Users: 138494, Number of movies:27262


In [ ]:
df.isnull().sum()

movieId        0
title          0
genres         0
userId       534
rating       534
timestamp    534
dtype: int64

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20000797 entries, 0 to 20000796
Data columns (total 6 columns):
 #   Column     Dtype  
---  ------     -----  
 0   movieId    int64  
 1   title      object 
 2   genres     object 
 3   userId     float64
 4   rating     float64
 5   timestamp  object 
dtypes: float64(2), int64(1), object(3)
memory usage: 1.0+ GB


In [ ]:
pd.DataFrame(df.title.value_counts())

,title
Pulp Fiction (1994),67310
Forrest Gump (1994),66172
"Shawshank Redemption, The (1994)",63366
"Silence of the Lambs, The (1991)",63299
Jurassic Park (1993),59715
...,...
Rapture (Arrebato) (1980),1
"Education of Mohammad Hussein, The (2013)",1
Satanas (2007),1
Psychosis (2010),1


In [ ]:
gf =pd.DataFrame(df['title'].value_counts())
gf

,title
Pulp Fiction (1994),67310
Forrest Gump (1994),66172
"Shawshank Redemption, The (1994)",63366
"Silence of the Lambs, The (1991)",63299
Jurassic Park (1993),59715
...,...
Rapture (Arrebato) (1980),1
"Education of Mohammad Hussein, The (2013)",1
Satanas (2007),1
Psychosis (2010),1


In [ ]:
df.shape

(20000797, 6)

In [ ]:
movie_rating_avg = df.groupby('title')['rating'].mean().sort_values(ascending=False).reset_index().rename(columns={'rating':'Average_rating'})
movie_rating_avg.head()

,title,Average_rating
0,Small Roads (2011),5.0
1,Divorce (1945),5.0
2,The Beautiful Story (1992),5.0
3,Into the Middle of Nowhere (2010),5.0
4,The Sea That Thinks (2000),5.0


In [ ]:
movie_rating_count =df.groupby('title')['rating'].count().sort_values(ascending =False).reset_index().rename(columns={'rating':'Total_RatingCount'})
movie_rating_count

,title,Total_RatingCount
0,Pulp Fiction (1994),67310
1,Forrest Gump (1994),66172
2,"Shawshank Redemption, The (1994)",63366
3,"Silence of the Lambs, The (1991)",63299
4,Jurassic Park (1993),59715
...,...,...
27257,"North and South, Book I (1985)",0
27258,"North and South, Book II (1986)",0
27259,The Happy Road (1957),0
27260,Angele (1934),0


In [ ]:
movie_rate_count_avg = movie_rating_count.merge(movie_rating_avg,on='title')
movie_rate_count_avg.head()

,title,Total_RatingCount,Average_rating
0,Pulp Fiction (1994),67310,4.174231
1,Forrest Gump (1994),66172,4.029000
2,"Shawshank Redemption, The (1994)",63366,4.446990
3,"Silence of the Lambs, The (1991)",63299,4.177057
4,Jurassic Park (1993),59715,3.664741


In [ ]:

rare_movie = gf[gf['title'] <=10000].index
# rare_movies
movies_df =df[~df['title'].isin(rare_movie)]
movies_df

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.0,4.0,1999-12-11 13:36:47
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,6.0,5.0,1997-03-13 17:50:52
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,8.0,4.0,1996-06-05 13:37:51
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,10.0,4.0,1999-11-25 02:44:47
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,11.0,4.5,2009-01-02 01:13:41
...,...,...,...,...,...,...
19985698,114240,Aladdin (1992),Adventure|Animation|Children|Comedy|Fantasy,28195.0,4.0,2014-09-22 20:52:18
19985699,114240,Aladdin (1992),Adventure|Animation|Children|Comedy|Fantasy,51334.0,3.0,2014-09-23 15:53:39
19985700,114240,Aladdin (1992),Adventure|Animation|Children|Comedy|Fantasy,120575.0,2.5,2014-10-08 14:23:39
19985701,114240,Aladdin (1992),Adventure|Animation|Children|Comedy|Fantasy,124998.0,2.5,2014-09-20 22:16:14


In [ ]:
movieRating_feature= movies_df.pivot_table(index='title',columns='userId',values='rating').fillna(0)


In [ ]:
def recommendedMovie(movieRating_feature):
    mat_movie_features_array = csr_matrix(movieRating_feature.values)
    model_knn = NearestNeighbors(metric='cosine', algorithm='brute',n_jobs=-1)
    model_knn.fit(mat_movie_features_array)
    query_index =np.random.choice(movieRating_feature.shape[0])
    print(query_index)
    distances,indices =model_knn.kneighbors(movieRating_feature.iloc[query_index,:].values.reshape(1,-1), n_neighbors=6)
    print("Distances -->",distances," Indices -->",indices)

    print(distances.flatten())
    print(len(distances.flatten()))

    for i in range(0, len(distances.flatten())):
        if i == 0:
            print('Recommendations for {0}:\n'.format(movieRating_feature.index[query_index]))
        else:
            print('{0}: {1}, with distance of {2}:'.format(i, movieRating_feature.index[indices.flatten()[i]], distances.flatten()[i]))

In [ ]:
recommendedMovie(movieRating_feature)

18
Distances --> [[0.         0.39530396 0.46541516 0.49087097 0.54224831 0.5586124 ]]  Indices --> [[ 18  16  17  15 402 408]]
[0.         0.39530396 0.46541516 0.49087097 0.54224831 0.5586124 ]
6
Recommendations for Alien³ (a.k.a. Alien 3) (1992):

1: Alien: Resurrection (1997), with distance of 0.39530396466595263:
2: Aliens (1986), with distance of 0.46541515820507784:
3: Alien (1979), with distance of 0.49087096670297037:
4: Starship Troopers (1997), with distance of 0.5422483145227739:
5: Terminator, The (1984), with distance of 0.5586124024900323:


In [ ]:
def recommendedMovie(movieRating_feature,favoriteMovie):
    mat_movie_features_array = csr_matrix(movieRating_feature.values)
    model_knn = NearestNeighbors(metric='cosine', algorithm='brute',n_jobs=-1)
    model_knn.fit(mat_movie_features_array)

    query_index = movieRating_feature.index.get_loc(favoriteMovie)

    print(query_index)
    distances,indices =model_knn.kneighbors(movieRating_feature.iloc[query_index,:].values.reshape(1,-1), n_neighbors=6)
    for i in range(0, len(distances.flatten())):
        if i == 0:
            print('Recommendations for {0}:\n'.format(movieRating_feature.index[query_index]))
        else:
            print('{0}: {1}, with distance of {2}:'.format(i, movieRating_feature.index[indices.flatten()[i]], distances.flatten()[i]))

In [ ]:
recommendedMovie(movieRating_feature,'Star Wars: Episode V - The Empire Strikes Back (1980)')

399
Recommendations for Star Wars: Episode V - The Empire Strikes Back (1980):

1: Star Wars: Episode IV - A New Hope (1977), with distance of 0.20741613801828163:
2: Star Wars: Episode VI - Return of the Jedi (1983), with distance of 0.2156239425493356:
3: Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981), with distance of 0.266592703428847:
4: Indiana Jones and the Last Crusade (1989), with distance of 0.3287503456378781:
5: Terminator, The (1984), with distance of 0.32956188888246263:


## Cosine Similarity

In [ ]:
# prompt:

import numpy as np
def cosine_similarity(x, y):
  return np.dot(x, y) / (np.linalg.norm(x) * np.linalg.norm(y))
